In [1]:
%load_ext autoreload
%autoreload 2

from process_data import PubmedQueries

PubmedQueries().main()

# process_TAR()

c:\Users\Simon\miniconda3\envs\RP\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Searching PMIDs:   0%|          | 0/28862 [00:00<?, ?it/s]


TypeError: 'NoneType' object does not support item assignment

In [50]:
%load_ext autoreload
%autoreload 2

import pandas as pd

df = pd.read_json('data/sysrev.jsonl', lines=True)
df['source'] = 'sysrev'
df.rename(columns={'title': 'nl_query', 'query': 'bool_query'}, inplace=True)
df.to_json('data/sysrev_conv.jsonl', orient="records", lines=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [56]:
import json
import pandas as pd
from pathlib import Path

def combine_for_model(md: Path, out_base: Path):
    qs = sorted(d for d in md.rglob("q*") if d.is_dir())
    if not qs: return
    out_base.mkdir(parents=True, exist_ok=True)
    out = out_base / md.name

    mdf = pd.concat(
        ((pd.read_csv(q/"final_trec_result"/"results.rel", sep=r"\s+", header=None,
            names=["id","accuracy","f1","f3","recall"]).assign(prompt_id=q.name))
        for q in qs if (q/"final_trec_result"/"results.rel").exists()), ignore_index=True)
    gen = []
    for q in qs:
        p = q/"generation_output"/"step_0.jsonl"
        if not p.exists(): continue
        for line in p.open():
            obj = json.loads(line)
            obj["prompt_id"] = q.name
            obj["generated_query"] = obj["existing_prompts"]["user"][1]["content"]
            del obj["existing_prompts"]
            gen.append(obj)
    gdf = pd.DataFrame(gen).astype({"id": str})
    mdf["id"] = mdf["id"].astype(str)

    # merge & reorder
    df = gdf.merge(mdf, on=["prompt_id","id"], how="left")
    cols = ["id", "prompt_id", "accuracy","f1","f3","recall"]
    cols += [c for c in df if c not in cols]

    # write JSONL
    with out.with_suffix(".jsonl").open("w", encoding="utf-8") as f:
        for rec in df[cols].to_dict(orient="records"):
            f.write(json.dumps(rec, ensure_ascii=False) + "\n")

BASE_DIR = Path("data") / "result_n_runs" / "seed_collection_reproduce"
OUTPUT_DIR = Path("data") / "combined_outputs"

for model_dir in BASE_DIR.iterdir():
    if model_dir.is_dir() and "Wang" not in str(model_dir):
        print(f"Processing model: {model_dir.name}")
        combine_for_model(model_dir, OUTPUT_DIR)

Processing model: gpt-3.5-turbo-0125
Processing model: gpt-3.5-turbo-1106
Processing model: gpt-4-1106-preview
Processing model: gpt-4o-mini
Processing model: HuggingfaceH4
Processing model: meta-llama
Processing model: mistralai
Processing model: o1-2024-12-17
Processing model: open-mistral-7b
Processing model: open-mixtral-8x7b
Processing model: original
